In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load dataset
file_path = '/content/drive/MyDrive/fer2013.csv'
data = pd.read_csv(file_path)

In [ ]:
# Define the emotion classes
CLASS_LABELS = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']

In [ ]:
# Preprocess data
def preprocess_data(data):
    images = np.array([np.fromstring(pixels, sep=' ').reshape(48, 48, 1) for pixels in data['pixels']])
    images = images / 255.0  # Normalize
    labels = to_categorical(data['emotion'], num_classes=len(CLASS_LABELS))
    return images, labels

In [ ]:
# Split dataset
images, labels = preprocess_data(data)
x_train, x_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Build CNN model
def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(CLASS_LABELS), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Build CNN-LSTM model
def build_cnn_lstm():
    model = Sequential([
        TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, 48, 48, 1)),
        TimeDistributed(MaxPooling2D((2, 2))),
        TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
        TimeDistributed(MaxPooling2D((2, 2))),
        TimeDistributed(Flatten()),
        LSTM(64, return_sequences=False),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(CLASS_LABELS), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Train models
cnn_model = build_cnn()
cnn_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 90s 220ms/step - accuracy: 0.2741 - loss: 1.7882 - val_accuracy: 0.3985 - val_loss: 1.5660
Epoch 2/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 78s 199ms/step - accuracy: 0.4062 - loss: 1.5481 - val_accuracy: 0.4559 - val_loss: 1.4328
Epoch 3/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 77s 196ms/step - accuracy: 0.4436 - loss: 1.4507 - val_accuracy: 0.4700 - val_loss: 1.3792
Epoch 4/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.4710 - loss: 1.3810 - val_accuracy: 0.4860 - val_loss: 1.3359
Epoch 5/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 84s 201ms/step - accuracy: 0.4931 - loss: 1.3336 - val_accuracy: 0.4984 - val_loss: 1.3164
Epoch 6/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 78s 191ms/step - accuracy: 0.5122 - loss: 1.2870 - val_accuracy: 0.5153 - val_loss: 1.2768
Epoch 7/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 77s 195ms/step - accuracy: 0.5354 - loss: 1.2182 - val_accuracy: 0.5181 - val_loss: 1.2648
Epoch 8/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 81s 192ms/step - accuracy: 0.5549 - loss: 1

In [ ]:
cnn_lstm_model = build_cnn_lstm()
x_train_lstm = x_train.reshape(-1, 1, 48, 48, 1)
x_val_lstm = x_val.reshape(-1, 1, 48, 48, 1)
cnn_lstm_model.fit(x_train_lstm, y_train, validation_data=(x_val_lstm, y_val), epochs=10, batch_size=64)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 101s 248ms/step - accuracy: 0.2602 - loss: 1.7966 - val_accuracy: 0.3907 - val_loss: 1.5624
Epoch 2/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 138s 238ms/step - accuracy: 0.4178 - loss: 1.5107 - val_accuracy: 0.4616 - val_loss: 1.3932
Epoch 3/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 135s 222ms/step - accuracy: 0.4854 - loss: 1.3512 - val_accuracy: 0.4804 - val_loss: 1.3504
Epoch 4/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 143s 224ms/step - accuracy: 0.5341 - loss: 1.2277 - val_accuracy: 0.5086 - val_loss: 1.2901
Epoch 5/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 142s 224ms/step - accuracy: 0.5814 - loss: 1.1131 - val_accuracy: 0.5179 - val_loss: 1.2792
Epoch 6/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 144s 230ms/step - accuracy: 0.6457 - loss: 0.9681 - val_accuracy: 0.5259 - val_loss: 1.2908
Epoch 7/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 139s 223ms/step - accuracy: 0.6989 - loss: 0.8272 - val_accuracy: 0.5235 - val_loss: 1.3814
Epoch 8/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 142s 222ms/step - accuracy: 0.7744 -

In [ ]:
# Evaluate models
cnn_accuracy = cnn_model.evaluate(x_test, y_test, verbose=0)[1]
cnn_lstm_accuracy = cnn_lstm_model.evaluate(x_test.reshape(-1, 1, 48, 48, 1), y_test, verbose=0)[1]
print(f"CNN Accuracy: {cnn_accuracy}, CNN-LSTM Accuracy: {cnn_lstm_accuracy}")

CNN Accuracy: 0.5362184047698975, CNN-LSTM Accuracy: 0.5356612205505371


In [ ]:
# Save the best model
best_model = cnn_model if cnn_accuracy > cnn_lstm_accuracy else cnn_lstm_model
best_model.save('/content/drive/MyDrive/best_facial_expression_model.keras')


In [ ]:
# Real-time face detection and expression recognition
def real_time_prediction():
    model = tf.keras.models.load_model('/content/drive/MyDrive/best_facial_expression_model.keras')
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = gray_frame[y:y+h, x:x+w]
            face = cv2.resize(face, (48, 48)).reshape(1, 48, 48, 1) / 255.0
            prediction = model.predict(face)
            emotion = CLASS_LABELS[np.argmax(prediction)]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        cv2.imshow('Facial Expression Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

real_time_prediction()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
!cp /content/drive/MyDrive/best_facial_expression_model.keras /content/

In [ ]:
from google.colab import files
files.download('/content/best_facial_expression_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>